### 1. Settings

In [16]:
#####################################
##########  DEPENDECIES ############
#####################################

import os
import pickle
import numpy as np
from tqdm import tqdm # type: ignore
import pandas as pd
import copy

from datasets import load_dataset, DatasetDict, Dataset
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
from sklearn.model_selection import KFold # type: ignore
import evaluate

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import utils.prep as pr
import utils.eval as ev
import utils.inference as infer

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math

from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from scipy.sparse import hstack

tqdm.pandas()

os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB_DISABLED"] = "true"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

import torch
#####################################
############  CONSTANTS #############
#####################################
RS = 42

MODEL = "CodeT5"
TRAIN_N = 330
BATCH_SIZE = 15
DECODER_LENGTH = 20
ENCODER_LENGTH = 15

FULL_TRAIN_ARGS = {
    "TRAIN_N": TRAIN_N,
    "BATCH_SIZE": BATCH_SIZE,
    "DECODER_LENGTH": DECODER_LENGTH,
    "ENCODER_LENGTH": ENCODER_LENGTH,
    "MODEL": MODEL,
    "SEQ_TRAINER_ARGS": {
        "overwrite_output_dir": True,
        "num_train_epochs": [0, 1, 2, 3, 4, 7, 9],
        "num_train_epochs_cluster": 1,
        "do_train": True,
        "do_eval": True,
        "per_device_train_batch_size": 4,
        "per_device_eval_batch_size": 4,
        "learning_rate": 5e-5,
        "warmup_steps": 100,
        "weight_decay": 0.1,
        "label_smoothing_factor": 0.1,
        "predict_with_generate": True,
        "logging_steps": 100,
        "save_total_limit": 1,
        "save_strategy": "no",
        "logging_strategy": "epoch",
        "evaluation_strategy": "epoch",
        "load_best_model_at_end": False,
    },
}
FULL_TRAIN_ARGS["SEQ_TRAINER_ARGS"]["output_dir"] = f'reports/results'
FULL_TRAIN_ARGS["SEQ_TRAINER_ARGS"]["logging_dir"] = f'reports/logs'

model_name="Salesforce/codet5-base-multi-sum"
tokenizer = AutoTokenizer.from_pretrained(model_name, skip_special_tokens=False)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/home/RDC/zinovyee.hub/.conda/envs/ensemble/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### 2. Conala data. Preprocessing. Sampling as in the paper (further, random sampling)

In [2]:
dataset = pd.read_csv(f"../data/processed/conala/20240327/conala_clustered.csv")
dataset = dataset.drop("time_batch", axis=1)

test_4_examples = dataset[dataset["cluster"]==4].sample(frac=0.85, random_state=RS)
print("Cluster 4 obsevations: ", test_4_examples.shape)
test_non4_examples = dataset[dataset["cluster"]!=4].sample(n=156, random_state=RS)
print("Cluster not 4 obsevations: ", test_non4_examples.shape)

test_dataset = pd.concat([test_4_examples, test_non4_examples])
train_dataset = dataset[~dataset.index.isin(test_dataset.index)]
print("Train Data: ", train_dataset.shape)
print("Test Data: ", test_dataset.shape)

train_dataset = Dataset.from_pandas(train_dataset.sample(frac=1, random_state=RS).reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_dataset.sample(frac=1, random_state=RS).reset_index(drop=True))

# Cross Validation
folds = KFold(n_splits=3, random_state=RS, shuffle=True)
questions_list = np.array(list(set(train_dataset["question_id"])))
splits_obj = folds.split(questions_list)
splits = []
for i, (train_idxs, val_idxs) in enumerate(splits_obj):
    print(f"Fold {i}")
    splits.append([train_idxs, val_idxs])

test_data = pr.preprocess_dataset(test_dataset, tokenizer=tokenizer)
test_df = pd.DataFrame(test_data)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
rouge = evaluate.load('rouge')

Cluster 4 obsevations:  (344, 6)
Cluster not 4 obsevations:  (156, 6)
Train Data:  (2379, 6)
Test Data:  (500, 6)
Fold 0
Fold 1
Fold 2


Filter: 100%|██████████| 500/500 [00:00<00:00, 48470.01 examples/s]


Map: 100%|██████████| 499/499 [00:00<00:00, 2166.77 examples/s]


In [3]:
train_dataset

Dataset({
    features: ['question_id', 'intent', 'rewritten_intent', 'snippet', 'idx', 'cluster'],
    num_rows: 2379
})

In [4]:
test_dataset


Dataset({
    features: ['question_id', 'intent', 'rewritten_intent', 'snippet', 'idx', 'cluster'],
    num_rows: 500
})

In [5]:
fold_results = {}
for epoch_i, epoch_set in enumerate(sorted(FULL_TRAIN_ARGS["SEQ_TRAINER_ARGS"]["num_train_epochs"])):
    fold_results[epoch_set] = {}
    
for i, (train_idxs, val_idxs) in enumerate(splits):
    latest_run_epoch = 0
    for epoch_i, epoch_set in enumerate(sorted(FULL_TRAIN_ARGS["SEQ_TRAINER_ARGS"]["num_train_epochs"])):
        print(f"TRAINING EPOCH SET {epoch_set}")

        TRAIN_ARGS = copy.deepcopy(FULL_TRAIN_ARGS)
        FOLD_MODEL_PATH = "./tmp/"

        if epoch_set > 1: 
            TRAIN_ARGS["SEQ_TRAINER_ARGS"]["num_train_epochs"] = epoch_set - latest_run_epoch
        else:
            TRAIN_ARGS["SEQ_TRAINER_ARGS"]["num_train_epochs"] = epoch_set
        
        print(f'TRAINING EPOCHS {TRAIN_ARGS["SEQ_TRAINER_ARGS"]["num_train_epochs"]}')

        print(f"Fold {i}")
        fold_dataset = DatasetDict({
            "train": train_dataset.filter(lambda q_id: q_id["question_id"] in questions_list[train_idxs]),
            "validation": train_dataset.filter(lambda q_id: q_id["question_id"] in questions_list[val_idxs]),
        })
        fold_train = pr.preprocess_dataset(fold_dataset["train"], tokenizer=tokenizer)
        fold_val = pr.preprocess_dataset(fold_dataset["validation"], tokenizer=tokenizer)
        fold_df = pd.DataFrame(fold_val)

        if epoch_set > 1: 
            model = AutoModelForSeq2SeqLM.from_pretrained(FOLD_MODEL_PATH)
            print(f"LOADING MODEL {FOLD_MODEL_PATH}")
        else: 
            model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
            print(f"LOADING MODEL {model_name}")

        print(device)
        model.to(device)

        data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
        compute_metrics = ev.compute_metric_with_params(tokenizer) 

        if not os.path.exists(f'reports/'): 
            os.mkdir(f'reports/')

        training_args = Seq2SeqTrainingArguments(
                **TRAIN_ARGS["SEQ_TRAINER_ARGS"],
            )
        
        trainer = Seq2SeqTrainer(
            model=model,
            args=training_args,
            data_collator=data_collator,
            train_dataset=fold_train,
            eval_dataset=fold_val,
            tokenizer=tokenizer,
            compute_metrics=compute_metrics,
        )

        if epoch_set!=0:
            trainer.train()

        text = fold_val["input_sequence"]
        summaries = infer.generate_summary(text, model, tokenizer, TRAIN_ARGS["ENCODER_LENGTH"], TRAIN_ARGS["DECODER_LENGTH"])
        
        fold_df["prediction"] = summaries[1]
        fold_df["rouge"] = rouge.compute(predictions=fold_df["prediction"], 
                    references=fold_df["output_sequence"],
                    use_stemmer=True, 
                    use_aggregator=False,
                    rouge_types=["rouge1"])["rouge1"]
        
        fold_results[epoch_set][i] = fold_df

        print("FOLDS IN RESULTS ", fold_results[epoch_set].keys())
        
        ########## SAVE FOLD MODEL
        if not os.path.exists(FOLD_MODEL_PATH): 
            os.mkdir(FOLD_MODEL_PATH)

        trainer.save_model(FOLD_MODEL_PATH)

        latest_run_epoch = epoch_set

########## CONVERT TO DATAFRAME

for epoch_i, (epoch_set) in enumerate(fold_results.keys()): 
    
    for i, (k, f_df) in enumerate(fold_results[epoch_set].items()): 
        
        f_df['fold'] = k
        f_df['epoch_set'] = epoch_set

        if (epoch_i==0 and i==0): 
            cv_df = f_df.copy()
        else: 
            cv_df = pd.concat([cv_df, f_df])

########## SAVE THE FILE

with open('cv_df_check_drift.pickle', 'wb') as handle:
    pickle.dump(cv_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

TRAINING EPOCH SET 0
TRAINING EPOCHS 0
Fold 0


Map: 100%|██████████| 799/799 [00:00<00:00, 2192.33 examples/s]
/home/RDC/zinovyee.hub/.conda/envs/ensemble/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


LOADING MODEL Salesforce/codet5-base-multi-sum
cuda


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/home/RDC/zinovyee.hub/.conda/envs/ensemble/lib/python3.10/site-packages/accelerate/accelerator.py:444: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/home/RDC/zinovyee.hub/.conda/envs/ensemble/lib/python3.10/site-packages/transformers/utils/import_utils.py:616: FutureWarning: `torch._dynamo.external_utils.is_compiling` is deprecated. Use `torch.compiler.is_compiling` instead.
  return dynamo.is_compiling()


FOLDS IN RESULTS  dict_keys([0])
TRAINING EPOCH SET 1
TRAINING EPOCHS 1
Fold 0


...

LOADING MODEL ./tmp/
cuda


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/home/RDC/zinovyee.hub/.conda/envs/ensemble/lib/python3.10/site-packages/accelerate/accelerator.py:444: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Bleu,Brevity Penalty,Length Ratio,Translation Length,Reference Length
1,1.700700,3.492974,0.458500,0.214400,0.417200,0.417100,14.926000,0.217600,0.954600,0.955600,9764,10218
2,1.896800,3.494753,0.467000,0.220300,0.422100,0.421900,15.346500,0.222800,1.000000,1.002300,10241,10218


/home/RDC/zinovyee.hub/.conda/envs/ensemble/lib/python3.10/site-packages/transformers/utils/import_utils.py:616: FutureWarning: `torch._dynamo.external_utils.is_compiling` is deprecated. Use `torch.compiler.is_compiling` instead.
  return dynamo.is_compiling()
/home/RDC/zinovyee.hub/.conda/envs/ensemble/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


FOLDS IN RESULTS  dict_keys([0, 1, 2])


In [6]:
########## LOAD CV RESULTS

import pickle
with open('cv_df_check_drift.pickle', 'rb') as handle:
    cv_df = pickle.load(handle)

########## ADD ZERO SHOT

# model_name="Salesforce/codet5-base-multi-sum"
# tokenizer = AutoTokenizer.from_pretrained(model_name, skip_special_tokens=False)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# zs_df = cv_df.copy()
# zs_df = zs_df.drop_duplicates(["question_id", "input_sequence"])
# zs_df["epoch_set"] = 0
# zs_df["prediction"] = infer.generate_summary(list(zs_df["input_sequence"].values), model, tokenizer, FULL_TRAIN_ARGS["ENCODER_LENGTH"], FULL_TRAIN_ARGS["DECODER_LENGTH"])[1]
# zs_df["rouge"] = rouge.compute(predictions=zs_df["prediction"].values, 
#                     references=zs_df["output_sequence"].values,
#                     use_stemmer=True, 
#                     use_aggregator=False,
#                     rouge_types=["rouge1"])["rouge1"]
# cv_df = pd.concat([cv_df, zs_df])

# with open('cv_df_chekc.pickle', 'wb') as handle:
#     pickle.dump(cv_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [7]:
########## ROUGE PER SETTING

print("Mean")
print(cv_df.groupby("epoch_set")["rouge"].mean())

print("STD")
print(cv_df.groupby("epoch_set")["rouge"].std())


Mean
epoch_set
0    0.312683
1    0.473042
2    0.473410
3    0.472201
4    0.466596
7    0.472444
9    0.465552
Name: rouge, dtype: float64
STD
epoch_set
0    0.172770
1    0.191672
2    0.192754
3    0.189420
4    0.189238
7    0.192562
9    0.189637
Name: rouge, dtype: float64


### Step 2. Learn performance

In [8]:
def step_two(X_train, y_train, model, X_val=None, y_val=None,  save=False): 

    if model=="lr":
        reg = LinearRegression().fit(X_train, y_train)
    elif model =="svm": 
        reg = SVR().fit(X_train, y_train)
    elif model=="rf":
        reg = RandomForestRegressor.fit(X_train, y_train)
    elif model=="lgbm":
        reg = LGBMRegressor()
        reg.fit(X=X_train, y=y_train)
    elif model=="catboost":
        reg = CatBoostRegressor()
        reg.fit(X=X_train, y=y_train)

    if save:
        with open(f'./models/reg_{model}_drift.pkl','wb') as f:
            pickle.dump(reg, f)
        return f'./models/reg_{model}_drift.pkl'
    
    else:
        y_pred = reg.predict(X_val)
        y_pred[y_pred<0] = 0
        mae = mean_absolute_error(y_true=y_val, y_pred=y_pred)
        rmse = math.sqrt(mean_squared_error(y_true=y_val, y_pred=y_pred))
        return {"pred": y_pred, "mae": mae, "rmse": rmse}

In [9]:
cv_df.fold.value_counts()

fold
2    5677
0    5593
1    5383
Name: count, dtype: int64

In [10]:
t_models = ["lr", "svm", "lgbm", "catboost"]

results = {}

cv_df["perf_hat"] = 0


for test_fold in range(cv_df.fold.max()+1):
    print(test_fold)

    # Prepare the input data
    vectorizer = TfidfVectorizer()
    X_train_tfidf = vectorizer.fit_transform(cv_df.loc[cv_df.fold!=test_fold, "input_sequence"])
    X_train_column_sparse = pd.get_dummies(cv_df.loc[cv_df.fold!=test_fold, "epoch_set"], sparse=True).sparse.to_coo().tocsr()
    X_train = hstack([X_train_column_sparse, X_train_tfidf])
    y_train = cv_df.loc[cv_df.fold!=test_fold, "rouge"]
    
    X_val_tfidf = vectorizer.transform(cv_df.loc[cv_df.fold==test_fold, "input_sequence"])
    X_val_column_sparse = pd.get_dummies(cv_df.loc[cv_df.fold==test_fold, "epoch_set"], sparse=True).sparse.to_coo().tocsr()
    X_val = hstack([X_val_column_sparse, X_val_tfidf])
    y_val = cv_df.loc[cv_df.fold==test_fold, "rouge"]

    results[test_fold] = {}
    for model in t_models:
        print(model)
        preds_df = step_two(X_train=X_train,
                            y_train=y_train,
                            X_val=X_val,
                            y_val=y_val,
                            model=model)
        cv_df.loc[cv_df.fold==test_fold, f"{model}_perf_hat"] = preds_df["pred"]
        cv_df.loc[cv_df.fold==test_fold, f"perf_hat"] = preds_df["pred"]
        results[test_fold][model] = preds_df

cv_df = cv_df.reset_index(drop=True)

0
lr


/tmp/ipykernel_2579514/518064818.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.         0.13174871 0.25035101 ... 0.35635605 0.50252097 0.51923574]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  cv_df.loc[cv_df.fold==test_fold, f"perf_hat"] = preds_df["pred"]


svm
lgbm
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.531717 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5559
[LightGBM] [Info] Number of data points in the train set: 11060, number of used features: 426
[LightGBM] [Info] Start training from score 0.444930
catboost
Learning rate set to 0.059854
0:	learn: 0.1919070	total: 59.8ms	remaining: 59.7s
1:	learn: 0.1907067	total: 65.8ms	remaining: 32.9s
2:	learn: 0.1895748	total: 70ms	remaining: 23.3s
3:	learn: 0.1885966	total: 73.7ms	remaining: 18.4s
4:	learn: 0.1876253	total: 77.4ms	remaining: 15.4s
5:	learn: 0.1868192	total: 80.8ms	remaining: 13.4s
6:	learn: 0.1861008	total: 83.7ms	remaining: 11.9s
7:	learn: 0.1854029	total: 86.2ms	remaining: 10.7s
8:	learn: 0.1848180	total: 89.1ms	remaining: 9.81s
9:	learn: 0.1842906	total: 91.8ms	remaining: 9.09s
10:	learn: 0.1838076	total: 94.5ms	remaining: 8.49s
11:	learn: 0.1833115	total: 96.9ms	remaining: 7.9

458:	learn: 0.1424728	total: 1.2s	remaining: 1.41s
459:	learn: 0.1424408	total: 1.2s	remaining: 1.41s
460:	learn: 0.1424039	total: 1.2s	remaining: 1.41s
461:	learn: 0.1423565	total: 1.21s	remaining: 1.4s
462:	learn: 0.1423134	total: 1.21s	remaining: 1.4s
463:	learn: 0.1422792	total: 1.21s	remaining: 1.4s
464:	learn: 0.1422307	total: 1.21s	remaining: 1.4s
465:	learn: 0.1421953	total: 1.22s	remaining: 1.39s
466:	learn: 0.1421628	total: 1.22s	remaining: 1.39s
467:	learn: 0.1421240	total: 1.22s	remaining: 1.39s
468:	learn: 0.1420988	total: 1.22s	remaining: 1.38s
469:	learn: 0.1420664	total: 1.23s	remaining: 1.38s
470:	learn: 0.1419940	total: 1.23s	remaining: 1.38s
471:	learn: 0.1419589	total: 1.23s	remaining: 1.38s
472:	learn: 0.1419119	total: 1.23s	remaining: 1.37s
473:	learn: 0.1418802	total: 1.24s	remaining: 1.37s
474:	learn: 0.1418185	total: 1.24s	remaining: 1.37s
475:	learn: 0.1417773	total: 1.24s	remaining: 1.37s
476:	learn: 0.1417386	total: 1.25s	remaining: 1.36s
477:	learn: 0.14170

In [11]:
cv_df.shape

(16653, 18)

In [12]:
cv_df.groupby("epoch_set")["catboost_perf_hat"].mean()

epoch_set
0    0.315937
1    0.471530
2    0.470979
3    0.470937
4    0.470615
7    0.471142
9    0.468458
Name: catboost_perf_hat, dtype: float64

In [13]:
# rearrange the file

model_results = {}

for model in t_models:
    model_results[model]= {}
    model_results[model]["rmse"] = []
    model_results[model]["mae"] = [] 

    for fold in range(3):
    
        model_results[model]["mae"].append(results[fold][model]["mae"])
        model_results[model]["rmse"].append(results[fold][model]["rmse"])
    
    model_results[model]["rmse_avg"] = np.array(model_results[model]["rmse"]).mean()
    model_results[model]["mae_avg"] = np.array(model_results[model]["mae"]).mean()

for model in t_models:
    print(model)
    print("RMSE ", model_results[model]["rmse_avg"])
    print("MAE ",model_results[model]["mae_avg"])
    print("\n")

with open('cd_df_with_predictions_drift.pickle', 'wb') as handle:
    pickle.dump(cv_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

lr
RMSE  0.24244748985294537
MAE  0.18582123061297395


svm
RMSE  0.1928748382897817
MAE  0.1525911464104188


lgbm
RMSE  0.18818029641497044
MAE  0.14916980218976042


catboost
RMSE  0.1863354091873544
MAE  0.14783516283012266




In [14]:
with open('cd_df_with_predictions_drift.pickle', 'rb') as handle:
    cv_df = pickle.load(handle)

# TRAIN ON ALL PREDICTIONS AT ONCE

t_models = ["lr", "svm", "lgbm", "catboost"]

# Prepare the input data
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(cv_df.loc[:, "input_sequence"])
X_train_column_sparse = pd.get_dummies(cv_df.loc[:, "epoch_set"], sparse=True).sparse.to_coo().tocsr()
X_train = hstack([X_train_column_sparse, X_train_tfidf])
y_train = cv_df.loc[:, "rouge"]
    
with open("./models/vectorizer_drift.pkl", "wb") as file:
    pickle.dump(vectorizer, file, protocol=pickle.HIGHEST_PROTOCOL) 
      
for model in t_models:
    print(model)
    preds_df = step_two(X_train=X_train,
                        y_train=y_train,
                        model=model,
                        save=True)

lr
svm
lgbm
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 4.408669 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8211
[LightGBM] [Info] Number of data points in the train set: 16653, number of used features: 563
[LightGBM] [Info] Start training from score 0.447990


catboost
Learning rate set to 0.063852
0:	learn: 0.1947212	total: 13.2ms	remaining: 13.2s
1:	learn: 0.1933866	total: 19.9ms	remaining: 9.94s
2:	learn: 0.1921984	total: 25.3ms	remaining: 8.41s
3:	learn: 0.1908246	total: 30ms	remaining: 7.47s
4:	learn: 0.1898393	total: 34ms	remaining: 6.78s
5:	learn: 0.1888927	total: 37.7ms	remaining: 6.25s
6:	learn: 0.1880930	total: 41.2ms	remaining: 5.84s
7:	learn: 0.1873528	total: 44.5ms	remaining: 5.52s
8:	learn: 0.1867759	total: 47.8ms	remaining: 5.26s
9:	learn: 0.1860229	total: 51ms	remaining: 5.05s
10:	learn: 0.1854063	total: 54.7ms	remaining: 4.91s
11:	learn: 0.1849419	total: 58.1ms	remaining: 4.78s
12:	learn: 0.1844837	total: 61.4ms	remaining: 4.66s
13:	learn: 0.1840126	total: 64.5ms	remaining: 4.55s
14:	learn: 0.1836164	total: 67.7ms	remaining: 4.45s
15:	learn: 0.1831898	total: 71.2ms	remaining: 4.38s
16:	learn: 0.1828187	total: 74.5ms	remaining: 4.31s
17:	learn: 0.1825076	total: 77.9ms	remaining: 4.25s
18:	learn: 0.1822333	total: 81.3ms	remain